In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns 
import numpy as np 
import pylab

In [3]:
pylab.rcParams['figure.figsize'] = 10, 8
%matplotlib inline

# Read Data

In [7]:
df = pd.read_csv('./data/data_cleaned/combined_data_state_level_na_filled_from_2000.csv')

In [8]:
df.head()

,state,year,enroll,total_revenue,total_expenditure,avg_attendence_elem_secd_schools,teachers_elem_secd_schools,high_school_completers_18_to_24,bachelor_completers_over_25,pupil_teacher_ratio,pub_high_school_graduates,expenditure_per_pupil,edu_expenditure_percent_gov_income,population,gdp_per_capita,income_per_capita,known_property_crimes,unemployment_rate
0,Alabama,2000,730184.0,4885917,5003267,725212.0,48194.4,0.722477,0.190,15.354315,37819.0,5758.428628,0.250000,4452173.0,33284,24258,151127.0,0.045750
1,Alaska,2000,132822.0,1313510,1386904,122412.0,7880.4,0.769277,0.247,16.922491,6615.0,9668.161193,0.123966,627963.0,57184,31651,21074.0,0.063583
2,Arizona,2000,814807.0,5479168,5397194,782851.0,44438.4,0.692453,0.235,19.750846,38304.0,5478.359673,0.190000,5160586.0,39185,26232,271269.0,0.039750
3,Arkansas,2000,450751.0,2844855,2683400,422958.0,31947.4,0.754282,0.167,14.084370,27335.0,5627.818516,0.195000,2678588.0,31834,22784,97501.0,0.042750
4,California,2000,5941946.0,46628207,45829753,5957216.0,298021.4,0.707395,0.266,20.605279,309866.0,6400.553393,0.260000,33987977.0,48223,33095,1054010.0,0.049583


In [9]:
df.isnull().sum()

state                                  0
year                                   0
enroll                                 0
total_revenue                          0
total_expenditure                      0
avg_attendence_elem_secd_schools       0
teachers_elem_secd_schools             0
high_school_completers_18_to_24        0
bachelor_completers_over_25            0
pupil_teacher_ratio                    0
pub_high_school_graduates              0
expenditure_per_pupil                  0
edu_expenditure_percent_gov_income    16
population                             0
gdp_per_capita                         0
income_per_capita                      0
known_property_crimes                  0
unemployment_rate                      0
dtype: int64

# Extract percent change rate for each individual states

In [151]:
for i,s in enumerate(df.state.unique()):    
    df1= df[df.state == s].copy()
    df1 = df1.sort_values(by = 'year').reset_index(drop = True)
    df1[df1.columns[2:]] = df1[df1.columns[2:]].pct_change()
    if i == 0:
        df_change = df1
    else:
        df_change = pd.concat([df_new, df1])
df_change = df_change.dropna(how = 'any')

In [154]:
df_change.head()

,state,year,enroll,total_revenue,total_expenditure,avg_attendence_elem_secd_schools,teachers_elem_secd_schools,high_school_completers_18_to_24,bachelor_completers_over_25,pupil_teacher_ratio,pub_high_school_graduates,expenditure_per_pupil,edu_expenditure_percent_gov_income,population,gdp_per_capita,income_per_capita,known_property_crimes,unemployment_rate
1,Alabama,2001,-0.002262,-0.010842,0.016545,-0.031424,0.096828,0.010855,0.021214,-0.069452,-0.020713,0.050982,-0.040000,0.003473,0.000841,0.030629,-0.020162,0.114754
2,Alabama,2002,-0.002972,0.061166,0.012146,-0.001691,0.053547,0.011030,0.019765,-0.050638,-0.010862,0.045476,-0.029167,0.002788,0.022695,0.019359,0.106652,0.156863
3,Alabama,2003,0.002111,0.013161,0.029354,0.007431,0.026099,0.011175,0.018442,-0.032546,-0.004046,0.049758,-0.008584,0.005224,0.023101,0.040337,0.053853,0.021186
4,Alabama,2004,0.003459,0.030804,0.028560,0.000000,0.009371,0.011291,0.017232,-0.016785,-0.000580,0.025622,0.012987,0.006048,0.052991,0.070079,0.016491,-0.053942
5,Alabama,2005,-0.001473,0.099672,0.083371,0.000201,0.001326,0.011378,0.016127,-0.004643,0.027123,0.072912,0.000000,0.008625,0.023868,0.050439,-0.178198,-0.210526


In [155]:
df_change.isnull().sum()

state                                 0
year                                  0
enroll                                0
total_revenue                         0
total_expenditure                     0
avg_attendence_elem_secd_schools      0
teachers_elem_secd_schools            0
high_school_completers_18_to_24       0
bachelor_completers_over_25           0
pupil_teacher_ratio                   0
pub_high_school_graduates             0
expenditure_per_pupil                 0
edu_expenditure_percent_gov_income    0
population                            0
gdp_per_capita                        0
income_per_capita                     0
known_property_crimes                 0
unemployment_rate                     0
dtype: int64

District of Columnbia is got removed

In [157]:
len(df_change.state.unique())

50